# MySQL Relational Database


<img src="datamodel.png" alt="Data Model" style="width: 300px; float: right; margin-left: 20px; border: 1px solid">

This sample database has the hospitals, systems, services, and visits information we talked about in the slides.  The model (table names, columns, and relationships) is show to the right.

In this model, there are four tables:
* System Affiliations
* Hospitals
* ED Visits
* Hospital Services

In these examples, we're going to ask a few questions and answer them with both SQL and Python Pandas data processes.


## Setup

Create a connection to the MySQL database

In [1]:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [2]:
host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'
# database = ''

In [3]:
conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

## Give me a list of hospitals

In [4]:
pd.read_sql_query("SELECT * FROM hospitals", conn)

,hospital_name,city,system_name,beds
0,BJH,St. Louis,BJC,1243
1,Mercy STL,Ladue,Mercy,1120
2,MoBap,Ladue,BJC,443


# All the hospitals that are not affiliated with the Catholic church

Using SQL, we join the hospitals with system_affiliations tables to get a list of hospitals.

In [5]:
pd.read_sql_query("""
  SELECT h.*, s.affiliation
  FROM 
    hospitals h JOIN
    system_affiliations s ON h.system_name = s.system_name
  WHERE
    s.affiliation != 'Catholic'
""", conn)

,hospital_name,city,system_name,beds,affiliation
0,BJH,St. Louis,BJC,1243,Non-Religious
1,MoBap,Ladue,BJC,443,Non-Religious


## Read in all the DB tables into an array of Dataframes

1. Get a list of all of the tables
2. Read each table into a dataframe; and
3. Store that in a dictionary of dataframes

In [6]:
df = pd.read_sql_query("SHOW TABLES", conn)

In [7]:
df

,Tables_in_hds5210
0,ed_visits
1,hospital_services
2,hospitals
3,paulboal_ipv
4,system_affiliations


In [8]:
tables = {}

for n,t in pd.read_sql_query("SHOW TABLES", conn).iterrows():
    name = str(t['Tables_in_hds5210'])
    print(name)
    tables[name] = pd.read_sql("SELECT * FROM "+name,conn)

ed_visits
hospital_services
hospitals
paulboal_ipv
system_affiliations


In [9]:
tables

{'ed_visits':   hospital_name  year  ed_visits
 0           BJH  2016      72348
 1           BJH  2017      81221
 2     Mercy STL  2016      51932
 3     Mercy STL  2017      52221
 4         MoBap  2016      42983
 5         MoBap  2017      43921,
 'hospital_services':    hospital_name services
 0            BJH   Clinic
 1            BJH       ED
 2            BJH       IP
 3            BJH       OP
 4      Mercy STL       ED
 5      Mercy STL       IP
 6      Mercy STL       OP
 7          MoBap   Clinic
 8          MoBap       ED
 9          MoBap       IP
 10         MoBap       OP,
 'hospitals':   hospital_name       city system_name  beds
 0           BJH  St. Louis         BJC  1243
 1     Mercy STL      Ladue       Mercy  1120
 2         MoBap      Ladue         BJC   443,
 'paulboal_ipv': Empty DataFrame
 Columns: [hospital_name, year, ip_visits]
 Index: [],
 'system_affiliations':   system_name    affiliation
 0         BJC  Non-Religious
 1       Mercy       Catholic}

# All the hospitals that are not affiliated with the Catholic church

This time, using Pandas

In [10]:
hospitals = tables['hospitals'].merge(tables['system_affiliations'])

In [11]:
hospitals

,hospital_name,city,system_name,beds,affiliation
0,BJH,St. Louis,BJC,1243,Non-Religious
1,MoBap,Ladue,BJC,443,Non-Religious
2,Mercy STL,Ladue,Mercy,1120,Catholic


In [12]:
hospitals[hospitals['affiliation'] != 'Catholic']

,hospital_name,city,system_name,beds,affiliation
0,BJH,St. Louis,BJC,1243,Non-Religious
1,MoBap,Ladue,BJC,443,Non-Religious


## Hospitals with more than 43,000 ED visits each year

First in SQL

In [13]:
pd.read_sql_query("""
  SELECT h.*, v.*
  FROM 
    hospitals h JOIN
    ed_visits v ON h.hospital_name = v.hospital_name
  WHERE
    v.ed_visits > 43000
""", conn)

,hospital_name,city,system_name,beds,hospital_name,year,ed_visits
0,BJH,St. Louis,BJC,1243,BJH,2016,72348
1,BJH,St. Louis,BJC,1243,BJH,2017,81221
2,Mercy STL,Ladue,Mercy,1120,Mercy STL,2016,51932
3,Mercy STL,Ladue,Mercy,1120,Mercy STL,2017,52221
4,MoBap,Ladue,BJC,443,MoBap,2017,43921


In [ ]:
hospitals = tables['population'].merge(tables['system_affiliations'])

Now, let's use SQL to pivot that so that we can see 2016 and 2017 in two separate columns...

# ...

Come on.  Let's do it?

# ...

Oh.  That isn't supported.  :(

## Let's try with Pandas

In [14]:
visits = tables['hospitals'].merge(tables['ed_visits'])

In [15]:
visits

,hospital_name,city,system_name,beds,year,ed_visits
0,BJH,St. Louis,BJC,1243,2016,72348
1,BJH,St. Louis,BJC,1243,2017,81221
2,Mercy STL,Ladue,Mercy,1120,2016,51932
3,Mercy STL,Ladue,Mercy,1120,2017,52221
4,MoBap,Ladue,BJC,443,2016,42983
5,MoBap,Ladue,BJC,443,2017,43921


In [16]:
visits = visits[visits['ed_visits'] > 43000]

In [17]:
visits

,hospital_name,city,system_name,beds,year,ed_visits
0,BJH,St. Louis,BJC,1243,2016,72348
1,BJH,St. Louis,BJC,1243,2017,81221
2,Mercy STL,Ladue,Mercy,1120,2016,51932
3,Mercy STL,Ladue,Mercy,1120,2017,52221
5,MoBap,Ladue,BJC,443,2017,43921


In [18]:
pd.pivot_table(visits, 
               index=['hospital_name','city','system_name','beds'], 
               columns='year', 
               values='ed_visits').reset_index()

year,hospital_name,city,system_name,beds,2016,2017
0,BJH,St. Louis,BJC,1243,72348.0,81221.0
1,Mercy STL,Ladue,Mercy,1120,51932.0,52221.0
2,MoBap,Ladue,BJC,443,NaN,43921.0


## OK.  We can do it in SQL...

But can you figure out why this is a bad idea?

In [19]:
pd.read_sql_query("""
  SELECT 
    h.*, v.year, 
    CASE WHEN v.year = 2016 THEN v.ed_visits ELSE NULL END as visits_2016,
    CASE WHEN v.year = 2017 THEN v.ed_visits ELSE NULL END as visits_2017
  FROM 
    hospitals h JOIN
    ed_visits v ON h.hospital_name = v.hospital_name
  WHERE
    v.ed_visits > 43000
""", conn)

,hospital_name,city,system_name,beds,year,visits_2016,visits_2017
0,BJH,St. Louis,BJC,1243,2016,72348.0,NaN
1,BJH,St. Louis,BJC,1243,2017,NaN,81221.0
2,Mercy STL,Ladue,Mercy,1120,2016,51932.0,NaN
3,Mercy STL,Ladue,Mercy,1120,2017,NaN,52221.0
4,MoBap,Ladue,BJC,443,2017,NaN,43921.0


In [20]:
pd.read_sql_query("""
  SELECT 
    h.*, 
    MIN(CASE WHEN v.year = 2016 THEN v.ed_visits ELSE NULL END) as visits_2016,
    MIN(CASE WHEN v.year = 2017 THEN v.ed_visits ELSE NULL END) as visits_2017
  FROM 
    hospitals h JOIN
    ed_visits v ON h.hospital_name = v.hospital_name
  WHERE
    v.ed_visits > 43000
  GROUP BY 
    h.hospital_name, h.city, h.system_name, h.beds
""", conn)

,hospital_name,city,system_name,beds,visits_2016,visits_2017
0,BJH,St. Louis,BJC,1243,72348.0,81221
1,Mercy STL,Ladue,Mercy,1120,51932.0,52221
2,MoBap,Ladue,BJC,443,NaN,43921


## Now, let's create some new data from scratch

1. Create a dataframe
2. Use a name that includes your username so that it's unique
3. Write the df to a new table
4. Query your data back out

See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html

In [21]:
columns = ['hospital_name','year','ip_visits']
data = [
    ['BJH',2016,3124],
    ['Mercy STL',2016,4321],
    ['MoBap',2016,2783]
]
df = pd.DataFrame(data, columns=columns)

In [22]:
df

,hospital_name,year,ip_visits
0,BJH,2016,3124
1,Mercy STL,2016,4321
2,MoBap,2016,2783


In [23]:
import getpass
myname = getpass.getuser().split('-')[1]
myname

'paulboal'

In [26]:
df.to_sql(myname + '_ipv', conn, index=False)

In [27]:
ip_visits = pd.read_sql_query('SELECT * FROM ' + myname + '_ipv', conn)

In [28]:
ip_visits

,hospital_name,year,ip_visits
0,BJH,2016,3124
1,Mercy STL,2016,4321
2,MoBap,2016,2783


In [31]:
pd.read_sql_query('SHOW TABLES', conn)

,Tables_in_hds5210
0,ed_visits
1,hospital_services
2,hospitals
3,system_affiliations


## And we need to clean up after ourselves by dropping our table

In [30]:
with conn.connect() as c:
    c.execute('DROP TABLE IF EXISTS ' + myname + '_ipv')